In [4]:
# Read stock symbols from the file
with open('sp500_symbols.txt', 'r') as file:
    stock_symbols = [line.strip() for line in file.readlines()]
print(f"Loaded {len(stock_symbols)} stock symbols.")

Loaded 503 stock symbols.


In [ ]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import time

# Define the date range
end_date = datetime.now()
start_date = end_date - timedelta(days=730)  # 2 years ago

# Load S&P 500 symbols from the .txt file
with open('sp500_symbols.txt', 'r') as file:
    sp500_symbols = [line.strip() for line in file if line.strip()]

# Initialize a dictionary to hold the data
data = {}

# Fetch historical data
for symbol in sp500_symbols:
    try:
        # Download historical data
        stock_data = yf.download(symbol, start=start_date, end=end_date, auto_adjust=False)
        data[symbol] = stock_data
        print(f"Fetched data for {symbol}")
        
        # Delay to avoid rate limiting
        time.sleep(1)  # Adjust the delay as necessary
    except Exception as e:
        print(f"Could not fetch data for {symbol}: {e}")

# Combine data into a single DataFrame
combined_data = pd.concat(data, axis=1)

# Fill missing data if necessary (optional)
combined_data.fillna(method='ffill', inplace=True)  # Forward fill to handle missing values

# Save to a CSV file
combined_data.to_csv('sp500_historical_data.csv')

print("Data saved to sp500_historical_data.csv")

[*********************100%***********************]  1 of 1 completed


Fetched data for MMM


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Fetched data for AOS
Fetched data for ABT


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


...

[*********************100%***********************]  1 of 1 completed


Fetched data for CNP


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Fetched data for CF
Fetched data for CRL


KeyboardInterrupt: 